In [1]:
import sys
sys.path.append('../../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.envs.GPEnv import MultiEnv
from wip.Code.train_ac import make_dataset, backward_one
import time
from collections import OrderedDict



In [2]:

BATCH_SIZE = 512#64
SIM_TIME = 1
MAX_ITER = 10
USE_JIT = False
DIMS = 2

device = torch.device('cuda')

env = MultiEnv(BATCH_SIZE, MAX_ITER, device, dims=DIMS)

#torch.backends.cudnn.enabled = False




In [3]:
from Code.Networks import OuterWrapper
#model = torch.load('../../models/rsnn_gppred4')
model = torch.load('../../models/rsnn_gppred2d1')
model_memory = model.layers.mem_loop.model.model

#model = OuterWrapper(model, device, USE_JIT)
model_memory = OuterWrapper(model_memory, device, USE_JIT, two_dim=True)

In [4]:
from Code.Networks import Selector, DynNetwork, OuterWrapper, LSTMWrapper, ReLuWrapper, DummyNeuron, make_SequenceWrapper
from Code.NewNeurons import SeqOnlySpike, CooldownNeuron

architecture = OrderedDict([
    ('input', 128),
    ('layer1', [['input'], ReLuWrapper(256), nn.Linear]),
    ('layer2', [['layer1'], ReLuWrapper(128), nn.Linear]),
    ('output', [['layer2'], DummyNeuron(1), nn.Linear]),
])

p_architecture = OrderedDict([
    ('input', 128),
    ('layer1', [['input'], ReLuWrapper(256), nn.Linear]),
    ('layer2', [['layer1'], ReLuWrapper(128), nn.Linear]),
    ('output', [['layer2'], DummyNeuron(DIMS), nn.Linear]),
])


In [5]:
#144, 150, 137, 150

v_model = OuterWrapper(DynNetwork(architecture), device, USE_JIT)
p_model = OuterWrapper(DynNetwork(p_architecture), device, USE_JIT)

#model = (OuterWrapper(DynNetwork(architecturelstm), device, True))



In [6]:
params = list(v_model.parameters())+list(p_model.parameters())
optimizer = optim.Adam(params, lr=1e-4)#0.00001



In [19]:
def train_dataset(num_batches, num_epochs, var, gamma):
    obs, mem, value, advantage, actions, avg_reward = make_dataset(num_batches, BATCH_SIZE, MAX_ITER, v_model, p_model, model_memory, device, env, var, gamma, DIMS)
    print('Avarage Reward: ', avg_reward)
    print('Action Variance: ', actions.var(0).mean())
    print('Avarage Start Value: ', value[0].mean())
    for e in range(num_epochs):
        start = time.time()
        idc = torch.randperm(num_batches*BATCH_SIZE*MAX_ITER, device=device)
        sum_v = 0
        sum_p = 0
        for i in range(num_batches*MAX_ITER):
            base = i*BATCH_SIZE
            batch_mem = mem.view(-1, 128)[idc[base:base + BATCH_SIZE]]
            batch_value = value.view(-1, 1)[idc[base:base + BATCH_SIZE]]
            batch_advantage = advantage.view(-1, 1)[idc[base:base + BATCH_SIZE]]
            batch_action = actions.view(-1, DIMS)[idc[base:base + BATCH_SIZE]]
            v_model.zero_grad()
            p_model.zero_grad()
            lossv, lossp = backward_one(batch_action, batch_mem, batch_value, batch_advantage, v_model, p_model, var, device)
            sum_v += lossv
            sum_p += lossp
            optimizer.step()
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
        print(sum_v / (num_batches*MAX_ITER), sum_p / (num_batches*MAX_ITER), time.time()-start)
            #if i%10 == 0:
                #print(loss.item(), (loss/targets.view(-1).var()).item(), i)

In [20]:
for i in range(100):
    print('Bigstep: ', i)
    train_dataset(200, 5, 0.05, 1)
    #env.render()

Bigstep:  0
Avarage Reward:  tensor(1.1222, device='cuda:0')
Action Variance:  tensor(0.0210, device='cuda:0')
Avarage Start Value:  tensor(1.1482, device='cuda:0')
0.054459097771905364 -0.6610714540928602 14.402873992919922
0.05415115465875715 -0.6652176461070776 14.231192350387573
0.0539537689331919 -0.6662207016497851 14.045667171478271
0.05383285529445857 -0.6668971728384495 14.15630578994751
0.053740212516859176 -0.6675251735001803 14.348989009857178
Bigstep:  1
Avarage Reward:  tensor(1.2307, device='cuda:0')
Action Variance:  tensor(0.0434, device='cuda:0')
Avarage Start Value:  tensor(1.1856, device='cuda:0')
0.060487560283392666 -0.6935089297294617 13.875427007675171
0.06018289816658944 -0.6974934040158987 14.005084753036499
0.05997672463487834 -0.6986214301139116 14.179445743560791
0.059879340541549024 -0.6996264088451862 14.058367252349854
0.05977027854602784 -0.7001919042170047 13.949779748916626
Bigstep:  2
Avarage Reward:  tensor(1.2347, device='cuda:0')
Action Variance: 

In [ ]:
env.render()

In [ ]:
model_memory

In [ ]:
env.reset()

In [ ]:
testenv = MultiEnv(BATCH_SIZE, MAX_ITER, device)

In [ ]:
cobs = testenv.reset()
cmem, h = model_memory(cobs, None)
v_old = 0
r_sum = 0
for k in range(MAX_ITER):
    action = torch.tensor([k / (MAX_ITER-1)], dtype=torch.float, device=device).expand(BATCH_SIZE, 1)
    cobs, reward, _ = testenv.step(action)
    r_sum += reward.mean()
print(r_sum)

In [ ]:
testenv.render()

In [17]:
testenv = MultiEnv(BATCH_SIZE, MAX_ITER, device, dims=DIMS)

cobs = testenv.reset()
cmem, h = model_memory(cobs, None)
v_old = 0
r_sum = 0
for k in range(MAX_ITER):
    action = torch.rand([BATCH_SIZE, 2], dtype=torch.float, device=device)
    cobs, reward, _ = testenv.step(action)
    r_sum += reward.mean()
print(r_sum)

tensor(1.4234, device='cuda:0')
